In [1]:
import pandas as pd
import gc

train_df = pd.read_csv('./train_transaction.csv')
train_idt_df = pd.read_csv('./train_identity.csv')
test_df = pd.read_csv('./test_transaction.csv')
test_idt_df = pd.read_csv('./test_identity.csv')
test_df2 = pd.read_csv('./test_transaction.csv')

train_df = pd.merge(train_df, train_idt_df, on='TransactionID', how='left')
test_df = pd.merge(test_df, test_idt_df, on='TransactionID', how='left')

train_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [2]:
train_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [3]:
train_len = len(train_df)
test_len = len(test_df)
all_df = train_df.append(test_df).reset_index()

c = [
    'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30',
    'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37',
    'id_38', 'DeviceType', 'DeviceInfo'
]

for col in c:
    all_df[col] = all_df[col].astype('category').cat.codes    

c:\users\sonse\miniconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
all_df = all_df.fillna(0)
all_df = pd.get_dummies(all_df)

train_df = all_df[:train_len]
test_df = all_df[train_len:]

del all_df
gc.collect()

112

In [5]:
big_top_train_cols = [col for col in train_df.columns if train_df[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_test_cols = [col for col in test_df.columns if test_df[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
many_null_train_cols = [col for col in train_df.columns if train_df[col].isnull().sum() / train_df.shape[0] > 0.9]
many_null_test_cols = [col for col in test_df.columns if test_df[col].isnull().sum() / test_df.shape[0] > 0.9]

cols_to_drop = list(set(big_top_train_cols + big_top_test_cols + many_null_train_cols + many_null_test_cols))
cols_to_drop.remove('isFraud')

train_df = train_df.drop(cols_to_drop, axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)

In [6]:
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold

folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1001)

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['isFraud', 'TransactionID', 'index']]

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['isFraud'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['isFraud'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['isFraud'].iloc[valid_idx]

    # LightGBM parameters found by Bayesian optimization
    clf = LGBMClassifier(
        nthread=4,
        n_estimators=1400,
        learning_rate=0.02,
        num_leaves=34,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.041545473,
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775,
        silent=-1,
        verbose=-1, )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
            eval_metric='auc', verbose=200, early_stopping_rounds=200)
    
    sub_preds = clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1]
    break

Training until validation scores don't improve for 200 rounds.
[200]	training's auc: 0.906808	training's binary_logloss: 0.0873171	valid_1's auc: 0.909939	valid_1's binary_logloss: 0.0862385
[400]	training's auc: 0.921549	training's binary_logloss: 0.0804295	valid_1's auc: 0.922098	valid_1's binary_logloss: 0.080185
[600]	training's auc: 0.930526	training's binary_logloss: 0.0760734	valid_1's auc: 0.928616	valid_1's binary_logloss: 0.0766218
[800]	training's auc: 0.937086	training's binary_logloss: 0.0727198	valid_1's auc: 0.933491	valid_1's binary_logloss: 0.0740141
[1000]	training's auc: 0.942214	training's binary_logloss: 0.0699101	valid_1's auc: 0.937249	valid_1's binary_logloss: 0.0719058
[1200]	training's auc: 0.946405	training's binary_logloss: 0.0675406	valid_1's auc: 0.940491	valid_1's binary_logloss: 0.070099
[1400]	training's auc: 0.950272	training's binary_logloss: 0.0653078	valid_1's auc: 0.943466	valid_1's binary_logloss: 0.0684851
Did not meet early stopping. Best iterat

In [7]:
output = pd.DataFrame({'TransactionID': test_df2.TransactionID, 'isFraud': sub_preds})
output.to_csv('submission.csv', index=False)
